# make a very simple LSTM model with keras

see how long this will take to run for 7t data with a not too complicated model and what kind of resources we are talking about - single run single subject..

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler
import pandas as pd
import nibabel as nb
import hrf_tools
#from analysis import load_data_HCP_MMP

In [2]:
def load_data_HCP_MMP(subject,feature,n_movies):
    # Inputs: subject = HCP id eg 100610
    #         feature='mfs'
    #         n_movies is a list of movie indices 1 thru 4
    # Returns: X feature data (2D; time x feature)
    #          Y brain data (2D; time x grayordinate)
    from sklearn.preprocessing import StandardScaler
    from scipy.signal import resample

    scaler = StandardScaler()

    y_l=[]
    x_l=[]
    stim = ['tfMRI_MOVIE1_7T_AP','tfMRI_MOVIE2_7T_PA','tfMRI_MOVIE3_7T_PA','tfMRI_MOVIE4_7T_AP']
    stim_feat = ['7T_MOVIE1_CC1_v2', '7T_MOVIE2_HO1_v2', '7T_MOVIE3_CC2_v2', '7T_MOVIE4_HO2_v2']
    slice_starts = [
        [20,284, 526,734],
        [20,267,545],
        [20,221,425,649],
        [20,272,522]]
    slice_stops =  [
        [264,506,714,798],
        [247,525,795],
        [201,405,629,792],
        [252,502,777]]
    for i in n_movies:
        i=i-1
        exclude_final=slice_stops[i][-1] #trim the final movie since it is in all scans
        #load brain image
        im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/parcellations/parcellated/sub{str(subject)}_{stim[i]}.ptseries.nii'
        img = nb.load(im_file)
        img_y = img.get_fdata()
        img_y = scaler.fit_transform(img_y)

        #load feature
        feat_x = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_{feature}.npy')
        feat_x = resample(feat_x, img_y.shape[0], axis=0) #resample to 1hz for now 
        #feat_x=feat_x.T
        #trim final movies
        img_y = img_y[:exclude_final,:]
        feat_x = feat_x[:exclude_final,:]
        y_l.append(img_y)
        x_l.append(feat_x)
    Y=np.vstack(y_l)
    X=np.vstack(x_l)
    #X = scaler.fit_transform(X)
    #vertex_info = hcp.get_HCP_vertex_info(img)
    return X,Y

In [3]:
print(np.__version__)

1.19.5


In [4]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10957096950636860581
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23552872960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1229633727219759524
physical_device_desc: "device: 0, name: Quadro RTX 6000, pci bus id: 0000:5e:00.0, compute capability: 7.5"
]


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

### load data

In [7]:
subject=100610
feature='as_scores'
n_movies=[1,2,3]
X,Y = load_data_HCP_MMP(subject,feature,n_movies)

X_test,Y_test = load_data_HCP_MMP(subject,feature,[4])

X = hrf_tools.apply_optimal_hrf_10hz(X,1)
X_test = hrf_tools.apply_optimal_hrf_10hz(X_test,1)

In [9]:
X.shape

(2385, 521)

In [29]:
X_val = X[:-500,:]
Y_val = Y[:-500,:]
X_train = X[-500:,:]
Y_train = Y[-500:,:]

In [28]:
Y[-500:,:].shape

(500, 360)

In [11]:
# Batch size
batch_size = 64
time_steps = 10
input_units = 521
input_shape = (time_steps,input_units)

In [30]:
#https://keras.io/api/preprocessing/timeseries/
dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_train, 
                                                     targets = Y_train, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)
dataset_val = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_val, 
                                                     targets = Y_val, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

dataset_test = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_test, 
                                                     targets = Y_test, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

### design a simple(?) lstm model

In [31]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
#model.add(layers.Embedding(input_dim=1024, output_dim=1024))

model.add(keras.Input(shape=input_shape)) #omit beacuse of bug
model.add(layers.Dropout(0.1))
# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(input_units))#,batch_input_shape=(batch_size, time_steps, input_units)))
model.add(layers.Dense(521,activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(521,activation='relu'))
model.add(layers.Dropout(0.1))
#model.add(layers.Dense(360))

#model.add(layers.UpSampling1D(size=10))
#model.add(layers.Dense(17049))
#model.add(layers.Dense(128))
model.add(layers.Dense(360,activation='relu'))
#model.add(layers.Dense(360))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_6 (Dropout)          (None, 10, 521)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 521)               2173612   
_________________________________________________________________
dense_6 (Dense)              (None, 521)               271962    
_________________________________________________________________
dropout_7 (Dropout)          (None, 521)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 521)               271962    
_________________________________________________________________
dropout_8 (Dropout)          (None, 521)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 360)              

In [32]:
# #dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(x_train,0), np.expand_dims(y_train,0)))
# dataset = tf.data.Dataset.from_tensor_slices((X_hrf, Y))

# dataset.element_spec

In [33]:


# # Buffer size to shuffle the dataset
# # (TF data is designed to work with possibly infinite sequences,
# # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# # it maintains a buffer in which it shuffles elements).
# BUFFER_SIZE = 10000

# dataset = (
#     dataset
#     .shuffle(BUFFER_SIZE)
#     .batch(batch_size, drop_remainder=True)
#     .prefetch(tf.data.experimental.AUTOTUNE))

# dataset

In [34]:
#simple_rnn = layers.SimpleRNN(18, return_sequences=True, return_state=True)

In [35]:
#whole_sequence_output, final_state = simple_rnn(np.expand_dims(x_train,0))

In [208]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, restore_best_weights=True, patience=25)

model.compile(
    loss=keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    optimizer="adam"
)

In [219]:
history = model.fit(dataset,epochs = 1000, validation_data=dataset_val, callbacks = [es])

Epoch 1/1000
38/38 [==============================] - 1s 26ms/step - loss: 0.6114 - val_loss: 0.5919
Epoch 2/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6119 - val_loss: 0.5903
Epoch 3/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6133 - val_loss: 0.5958
Epoch 4/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6126 - val_loss: 0.5966
Epoch 5/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6146 - val_loss: 0.5937
Epoch 6/1000
38/38 [==============================] - 1s 25ms/step - loss: 0.6108 - val_loss: 0.5945
Epoch 7/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6165 - val_loss: 0.5909
Epoch 8/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6127 - val_loss: 0.5865
Epoch 9/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6106 - val_loss: 0.5899
Epoch 10/1000
38/38 [==============================] - 1s 24ms/step - loss: 0.6135 - val_lo

In [202]:
model.save('../outputs/mmp_earlystop_00511.pb')

INFO:tensorflow:Assets written to: ../outputs/mmp_earlystop_00511.pb/assets


INFO:tensorflow:Assets written to: ../outputs/mmp_earlystop_00511.pb/assets


In [33]:
model = keras.models.load_model('../outputs/model.pb')

In [50]:
dataset_test = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_test, 
                                                     targets = Y_test, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [203]:
model.evaluate(dataset_test)

12/12 [==============================] - 0s 12ms/step - loss: 1.0640


1.0639604330062866

In [204]:
Y_predicted = model.predict(dataset_test)
# it is 360 long not 380, so it must be only cortex
Y_predicted.shape

(768, 360)

In [205]:
import npp
from sklearn.metrics import r2_score
r2 = r2_score(Y_test[9:,:], Y_predicted,multioutput='raw_values')
r = npp.mcorr(Y_test[9:,:], Y_predicted)
r = np.append(r, np.zeros(20))
r2 = np.append(r2, np.zeros(20))

r.shape

In [222]:
r2 = np.append(r2, np.zeros(20))


In [187]:
import nibabel as nib
import nilearn.plotting as plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import hcp_utils as hcp


(380,)

In [223]:
from analysis import plot_32k_results
plot_32k_results(hcp.unparcellate(r2, hcp.mmp),'r2',100610,'as_scores','HCP_MMP_LSTM10_e511_r2')

[-0.60541544 -0.58934379 -0.55266707 -0.50050411 -0.43722121 -0.43558113
 -0.41723345 -0.41123071 -0.40820145 -0.40738757 -0.400753   -0.3961009
 -0.3959338  -0.39348458 -0.39226126 -0.38949863 -0.38928398 -0.38773321
 -0.38592725 -0.3858073  -0.38297276 -0.37757682 -0.37720798 -0.3770676
 -0.36981633 -0.36366198 -0.3629101  -0.36234956 -0.35585319 -0.35490243
 -0.35271716 -0.35221839 -0.35212463 -0.3504849  -0.34967885 -0.34889434
 -0.34572595 -0.34560158 -0.34537841 -0.34528762 -0.33808161 -0.33682879
 -0.33605215 -0.3334054  -0.33048122 -0.32553797 -0.3253692  -0.32113236
 -0.32099961 -0.31846672 -0.3184623  -0.31648036 -0.31632305 -0.31533863
 -0.31523559 -0.31514684 -0.31408676 -0.31331144 -0.30981342 -0.30578311
 -0.29900258 -0.29684165 -0.29563076 -0.2955406  -0.29374022 -0.29264572
 -0.28788534 -0.28646566 -0.28299045 -0.28288196 -0.28215919 -0.28147888
 -0.28112779 -0.28049498 -0.28011843 -0.2764798  -0.27533344 -0.27441443
 -0.27425846 -0.27261367 -0.27164958 -0.26756589 -0.2

## compare to encoding model simple ridgecv

In [210]:
from analysis import simple_ridgeCV

In [ ]:
scores_mean,corr_mean,weights_mean = simple_ridgeCV

In [211]:
subject=100610
feature='as_scores'
n_movies=[1,2,3,4]
X,Y = load_data_HCP_MMP(subject,feature,n_movies)

X = hrf_tools.apply_optimal_hrf_10hz(X,1)


In [214]:
scores_mean,corr_mean,weights_mean = simple_ridgeCV(X,Y)

In [220]:
corr_mean = np.append(corr_mean, np.zeros(20))
plot_32k_results(hcp.unparcellate(corr_mean, hcp.mmp),'r',100610,'as_scores','HCP_MMP_ridgecv_r')

[-0.07379453 -0.0594298  -0.05796366 -0.05428356 -0.05242199 -0.05037978
 -0.049447   -0.03912499 -0.0343299  -0.02907795 -0.02699877 -0.02612969
 -0.02077062 -0.01689491 -0.01671508 -0.01365151 -0.01075032 -0.00795606
 -0.00239166  0.          0.00158645  0.00167508  0.00246106  0.00385166
  0.01373988  0.01395701  0.01607101  0.01708734  0.02032552  0.02125598
  0.02403139  0.02484643  0.02609002  0.03300244  0.03415697  0.03501747
  0.03591688  0.03648897  0.03809661  0.03885547  0.04095772  0.04195697
  0.04272116  0.04704868  0.04987922  0.0502834   0.0503497   0.05076438
  0.05140057  0.05612188  0.05691978  0.05892088  0.0591216   0.05988332
  0.06182481  0.06386767  0.06407604  0.06750781  0.06954963  0.06985272
  0.07025336  0.07210307  0.07325805  0.07348103  0.07398143  0.07451719
  0.07876466  0.07901132  0.07923717  0.08304534  0.08666907  0.08770605
  0.08849491  0.08945132  0.09123561  0.09129707  0.09207967  0.09313536
  0.09456293  0.09523509  0.09714286  0.1040952   0

In [221]:
scores_mean = np.append(scores_mean, np.zeros(20))
plot_32k_results(hcp.unparcellate(scores_mean, hcp.mmp),'r2',100610,'as_scores','HCP_MMP_ridgecv_r2')

[-3.84086601e-01 -3.58073785e-01 -3.39105902e-01 -3.29342852e-01
 -2.92308640e-01 -2.65493633e-01 -2.18180799e-01 -2.14697223e-01
 -2.09658447e-01 -2.01116373e-01 -1.97986011e-01 -1.97691755e-01
 -1.95378931e-01 -1.86271333e-01 -1.84490837e-01 -1.80938245e-01
 -1.79901183e-01 -1.79586407e-01 -1.75294187e-01 -1.71682008e-01
 -1.68478646e-01 -1.63028866e-01 -1.62737529e-01 -1.61268650e-01
 -1.59853997e-01 -1.58893601e-01 -1.57880894e-01 -1.54656457e-01
 -1.54417025e-01 -1.53443869e-01 -1.52695476e-01 -1.49738847e-01
 -1.49032309e-01 -1.46248910e-01 -1.45070043e-01 -1.43083167e-01
 -1.42728038e-01 -1.41936551e-01 -1.39327337e-01 -1.39053068e-01
 -1.38116434e-01 -1.38067119e-01 -1.37482099e-01 -1.34571376e-01
 -1.34104264e-01 -1.33612255e-01 -1.32643837e-01 -1.32297484e-01
 -1.28544002e-01 -1.27321714e-01 -1.21209504e-01 -1.18182741e-01
 -1.16651178e-01 -1.15489435e-01 -1.13360337e-01 -1.12175584e-01
 -1.08615063e-01 -1.08603649e-01 -1.08396269e-01 -1.05565430e-01
 -1.05540455e-01 -1.04590